## PS3

In [155]:
import numpy as np
from numpy.polynomial import Polynomial as P

def f(x):
    return np.exp(-x**2)
    
def findw(f,lower,upper):
    slope=(f(upper)-f(lower))/upper-lower
    A=(slope/2)*(upper**2)-(slope*upper-f(upper))*upper-((slope/2)*(lower**2)-(slope*lower-f(lower))*lower)
    return [slope/A, (-slope*upper+f(upper))/A] 

def integrate(f,w,lower,upper,N):
    nlower=P([-lower,w[1],w[0]/2]).roots()
    nupper=P([-upper,w[1],w[0]/2]).roots()
    print(nlower)
    print(nupper)
    lower=nlower[0]
    upper=nupper[0]
    inputs=(upper-lower)*np.random.rand(N)+lower
    p= [P([-i,w[1],w[0]/2]) for i in inputs]
    inverse_inputs=[]
    for i in p:
        if i.roots()[0]>=lower and i.roots()[0]<=upper:
            inverse_inputs.append(i.roots()[0])
        else: 
            inverse_inputs.append(i.roots()[1])
    inverse_inputs=np.array(inverse_inputs)
    outputsF=f(inverse_inputs)
    outputsW=w[0]*(inverse_inputs)+w[1]
    outputs=outputsF/outputsW
    return (upper-lower)*outputs.sum()/N  
    
l=np.arange(0,3,1)    
I=0
for i in range (0,len(l)-1):
    w=findw(f,l[i],l[i+1])
    I+=integrate(f,w,l[i],l[i+1],100)
    print(I)
print(I)

[ 0.          3.16395341]
[ 1.          2.16395341]
0.750087317436
[ 0.74023304  3.2909483 ]
[ 2.01559067-0.89973972j  2.01559067+0.89973972j]
(1.39501301033+0.315708999417j)
(1.39501301033+0.315708999417j)
